### FE - 680 Assignment 2

Instructor: Dragos Bozdog (PhD)

Completed By: Riley Heiman


In [1]:
import os
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np   
from scipy.stats import norm 

In [2]:
print(os.getcwd())  

d:\Documents\Stevens\FE-680\HW2


### Problem 1

What is the value of a European swap option that gives the holder the right to enter into a 6-year annual-pay swap in 3 years where a fixed rate of 3.8% is paid and LIBOR is received? 

The swap principal is $5 million. Assume that the yield curve is flat at 3.7% per annum with annual compounding and the volatility of the swap rate is 25%



Recall the following: (Page 685, 708 in Hull)

$$LA[s_0 N(d_1) - s_k N(d_2)]$$

Where: 

- $ A = \frac{1}{m} \sum_{i = 1}^{mn} P(0,T_i)$

    - m = number of payments per year 

    - $P(0,T_i) = $ Discount factor for the mn payoffs 

- $ L =$ notional principal

- $s_k = $ "fixed rate"

- $s_0 = $ "forward" swap rate at time zero. by eq (28.23)

- $d_1  = \frac{ln(\frac{s_0}{s_K}) + \sigma^2 \frac{T}{2}}{\sigma \sqrt{T}}$

- $d_2 = d_1 - \sigma \sqrt{T}$

In [14]:
# These are inputs into the model! 

time_cadence = 'annual'
print("Time Cadence = " + time_cadence )


L = 5000000
sk = .038   # fixed rate
interest_rate = .037
T = 3
sigma = .25


if time_cadence == 'semi-annual':
    tc = 2

elif time_cadence == 'annual':
    tc = 1


time_i = np.arange(3.0, 9.5, 1/tc)
print(time_i)

A = .5*np.sum( np.exp(- interest_rate*time_i ) )

so = tc*(np.exp((interest_rate/tc)) -1)  # convert continous interest to semi-annual

d1 = (np.log(so/sk) + sigma**2 * T/tc) / ( sigma * np.sqrt(T) )
#print(d1)
#.41428

d2 = d1 - sigma*np.sqrt(T)
#print(d2)
#-.0187

price = L*A*(so * norm.cdf(d1, loc=0, scale=1 )  - sk * norm.cdf(d2, loc=0, scale=1 ) )

price = np.round(price, decimals = 2)

print(' ')
print("The Price = " + str(price))



Time Cadence = annual
[3. 4. 5. 6. 7. 8. 9.]
 
The Price = 86946.92


---

### Problem 2

Find the value of a 4-year collar that guarantees that the min and max interest rates on a LIBOR-based loan (with quarterly resets) are 3.0% and 4.0% respectively. The LIBOR zero curve (continuously compounded) is currently flat at 3.6%. Use a flat volatility of 22%. Assume that the principal is $5 million.


$\:$

A collar
- long 1 put option and short 1 call option

- Buy 1 'floorlet' and sell 1 'caplet'
           

$$Collar \: Price = -1*(floorlet \: Price) + (caplet \: price)$$


#### Caplet

$$ L\delta_k P(0, t_{k+1})[F_k N(d_1) - R_k N(d_2)] \qquad (29.7) $$

Where: 
- $d_1 = \frac{ln(\frac{F_k}{R_k}) + \frac{\sigma^2_k t_k}{2} }{\sigma_k \sqrt{t_k}}$

- $d_2 = d_1 - \sigma_k \sqrt{t_k} $

- $F_k =$ Forward interest Rate

- $R_k = $ interest rate

- $\sigma_k = $ volatility of this forward interest rate

- $P(0, t_{k+1}) = $ risk-free discount factor

- $\delta_k = t_{k+1} - t_k$

#### Floorlet

$$ L\delta_k P(0, t_{k+1})[R_k N(-d_2) - F_k N(-d_1)] \qquad (29.8) $$

In [4]:

def CAPLET(Fk, L, R_k, t_k, t_k1, sigma_k, r ): 
    delta_k = t_k1 - t_k

    P_0_tk1 = np.exp(-r*t_k1)

    d1 = (np.log( Fk / R_k ) + (sigma_k**2 * t_k)/2) / sigma_k * np.sqrt(t_k)

    d2 = d1 - sigma_k * np.sqrt(t_k)

    caplet = (L * delta_k * P_0_tk1) * (Fk * norm.cdf(d1 , loc = 0 , scale = 1 ) - R_k * norm.cdf(d2 , loc = 0 , scale = 1 ))
    return(caplet)



In [5]:

def FLOORLET(Fk, L, R_k, t_k, t_k1, sigma_k, r ): 
    delta_k = t_k1 - t_k

    P_0_tk1 = np.exp(-r*t_k1)

    d1 = (np.log( Fk / R_k ) + (sigma_k**2 * t_k)/2) / sigma_k * np.sqrt(t_k)

    d2 = d1 - sigma_k * np.sqrt(t_k)

    floorlet = (L * delta_k * P_0_tk1) * (R_k * norm.cdf(-d2 , loc = 0 , scale = 1 ) - Fk * norm.cdf(-d1 , loc = 0 , scale = 1 ))
    return(floorlet)


In [6]:


Fk = .036
L = 5000000
R_k = .03 # for cap?   4% for floor?
t_k = 0
t_k1 = 4
sigma_k = .22
r = .036


caplet_price = CAPLET(Fk, L, R_k, t_k, t_k1, sigma_k, r )
caplet_price



51953.264883552285

In [7]:
R_k = .04

floorlet_price = FLOORLET(Fk, L, R_k, t_k, t_k1, sigma_k , r )
floorlet_price 


34635.50992236823

In [8]:
collar = -1*floorlet_price + caplet_price

print('The price of collar = $'  + str(  round (collar,2)))  

The price of collar = $17317.75


---

### Problem 3

The payoff from a derivative will occur in 7 years. It will equal the average of the one-year risk-free interest rates observed at times 1, 2, 3, 4, 5, 6, and 7 years applied to a principal of $5 mill. The risk-free yield curve is flat at 5% with annual compounding and the volatilities of all rates are 22%. Assume perfect correlation between all rates. What is the value of the derivative?

Timing adjustment:

$$E_{T^*} = E_T (V_T)exp [\frac{\rho_{VR} \sigma_V \sigma_R R_0 (T^* - T)}{1+ \frac{R_0}{m}}T] \qquad (30.4)$$


In [9]:
# This code was used to write the function below! 
rho = .4
sigma_v = .20
sigma_r = .18

T2 = 6
T1 = 5
delta_t = T2-T1
R_0 = .08

ET_VT = 1200


value = ET_VT * np.exp( (rho * sigma_v * sigma_r * R_0 *delta_t) / ( 1 + (R_0/1) ) )

value = value / ( 1 + R_0)**T2

print( value )



757.0105997309339


In [10]:
def PAYOFF_TIME_ADJUST(rho, sigma_v, sigma_r, T1, T2, R_0, ET_VT):
    VALUES = []

    time_intervals = np.arange(T1, T2, 1)
    l = len(time_intervals)

    for i in range(1, l):
        t_i = time_intervals[i]
        t_i_1 = time_intervals[i-1]
        delta_t = t_i - t_i_1

        value = np.exp( (rho * sigma_v * sigma_r * R_0 *delta_t) / ( 1 + (R_0/1) ) )
        VALUES.append(value)

    
    value = ET_VT*np.mean(VALUES) / ( 1 + R_0)**T2
    return(value)


In [11]:


rho = 1.0
sigma_v = .22
sigma_r = .22

T2 = 7
T1 = 1

R_0 = .05

ET_VT = 5000000

price = PAYOFF_TIME_ADJUST(rho, sigma_v, sigma_r, T1, T2, R_0, ET_VT)

print("The Price = $" + str( round( price , 2 ) ) )  


The Price = $3561605.85


---

### Problem 4

Assume that the risk free-rate yield curve is flat at 5% (with continuous compounding). The payoff from a derivative occurs in 5 years. It is equal to the 6-year rate minus the 2-year rate at this time, applied to a principal of $1 million with both rates being continuously compounded. Calculate the value of the derivative. Assume that the volatility for all rates is 30%. 

$\:$


**Feedback from Dr. Bozdog:** (Thank you!)

*The payoff of the derivative is equal to 6-year rate minus 2-year rate. So, if the forward bond price equals the expected bond price, then what is the expected yield on the bond with maturity in 6 years and in 2 years?*

$\:$ 

#####  Convexity Adjustment


$$ E_T (y_T) = y_0 - \frac{1}{2} y_0^2 \sigma_y^2 T \frac{G^{''} (y_0)}{G^{'} (y_0)} \qquad (30.1) $$

The value of the intrument is therefore: 

$$ P(0,T) L \tau [R_0 + \frac{R_0^2 \sigma_R^2 \tau T}{1+ R_0 \tau} ] $$

Where: 

- $P(0,T) = \frac{1}{(1+R_0)^T}$

In [12]:
L = 1000000
R0 = .05
sigma = .30
T = 2.0
T_star = 6.0

tau = T_star - T

P_0_T = 1.0 / ((1+R0)**T)   # is this a discount factor? 

price = P_0_T * L * tau * (  R0 + ( (R0**2 * sigma**2 * tau * T) / (1+ R0*tau)  )  )

print("The value = $" + str( round( price , 2 )  )  )


The value = $186848.07


--- 

### Final Remarks


I sincerely appreciate the opportunity to study fixed-income derivatives! I’m leaving some final thoughts on the chapter below for personal use. Feedback is always welcomed if time permits.  

Page 693 of Hull states: 

> The models presented in Chapter 29 for bond options are all examples of the “two-step” procedure. 
>
> This raises the issue is it always correct to value European style derivatives by using the “two-step” procedure. **The answer is no!**
> 
> For **nonstandard interest rate** derivatives, it is sometimes necessary to modify the two-step procedure so that an **adjustment** is made to the forward value of the variable in the first step.

$\:$

#### Questions to think about
- What does Hull mean by two step?
- How is “nonstandard interest rate” defined?
    - Is LIBOR nonstandard? 
    - What about stochastic interest rate models?

There are many models! 

One Factor Modles

-	Hull-White
-	Ho-Lee
-	CIR
-	Rendleman-Bartter model
-	Mertons model
-	Black-Derman-Toy
-	Black-Karasinski
-	Kalotay-Williams-Fabozzi model


Two Factor Models
-	Longstaff-Schwartz model
-	Chen model


Time may permit us to only master a few models. Based upon the 80-20 rule, which models are used most frequently? For example, when pricing American stock options, binomial tree, and monte carlo are used frequently even though more complex models exist.


--- 


### Reference

The equations listed in the notebook come from Hull

[1] John Hull. Options, Futures, and Other Derivatives. 11th Edition. Pearson. (ISBN:
978-0136939979)
